# RESTful APIs

Creating RESTful APIs with Django is a powerful way to build backend services that can interact with various frontends, mobile applications, or other systems. Django REST framework (DRF) is the most widely used library for creating REST APIs in Django.

### 1. What is REST?
REST (Representational State Transfer) is an architectural style for designing networked applications.


It uses standard HTTP methods like GET, POST, PUT, PATCH, and DELETE to interact with resources.
#### Key Concepts:
Resources: Represented as data objects (e.g., a user or a product).

Statelessness: Each request is independent and does not rely on server-side sessions.

JSON: Common format for data exchange.

### 2. Setting Up Django REST Framework (DRF)
Install DRF:

In [ ]:
pip install djangorestframework

In [ ]:
# settings.py
INSTALLED_APPS = [
    'rest_framework',
    'your_app_name',  # Your Django app
]


### 3. Creating a Basic API
Example: Building an API for a Book Resource

Define the Model:

In [ ]:
# models.py
from django.db import models

class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=100)
    published_date = models.DateField()
    isbn = models.CharField(max_length=13, unique=True)

    def __str__(self):
        return self.title



### Create a Serializer:
Serializers convert complex data types (e.g., model instances) into JSON and vice versa

In [ ]:
# serializers.py
from rest_framework import serializers
from .models import Book

class BookSerializer(serializers.ModelSerializer):
    class Meta:
        model = Book
        fields = '__all__'


### Create API Views:
You can use DRF's function-based views (FBVs) or class-based views (CBVs).

a) Function-Based Views (FBVs):

In [ ]:
# views.py
from rest_framework.decorators import api_view
from rest_framework.response import Response
from rest_framework import status
from .models import Book
from .serializers import BookSerializer

@api_view(['GET', 'POST'])
def book_list(request):
    if request.method == 'GET':
        books = Book.objects.all()
        serializer = BookSerializer(books, many=True)
        return Response(serializer.data)

    elif request.method == 'POST':
        serializer = BookSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

@api_view(['GET', 'PUT', 'DELETE'])
def book_detail(request, pk):
    try:
        book = Book.objects.get(pk=pk)
    except Book.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

    if request.method == 'GET':
        serializer = BookSerializer(book)
        return Response(serializer.data)

    elif request.method == 'PUT':
        serializer = BookSerializer(book, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    elif request.method == 'DELETE':
        book.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)


b) Class-Based Views (CBVs):


In [ ]:
from rest_framework.views import APIView
from rest_framework.response import Response
from rest_framework import status
from .models import Book
from .serializers import BookSerializer

class BookList(APIView):
    def get(self, request):
        books = Book.objects.all()
        serializer = BookSerializer(books, many=True)
        return Response(serializer.data)

    def post(self, request):
        serializer = BookSerializer(data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_201_CREATED)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


### Define URLs:

In [ ]:
# urls.py
from django.urls import path
from .views import book_list, book_detail

urlpatterns = [
    path('books/', book_list, name='book-list'),
    path('books/<int:pk>/', book_detail, name='book-detail'),
]


### 4. Advanced API Development
Using Generic Views

DRF provides generic views to handle common patterns.

In [ ]:
from rest_framework import generics
from .models import Book
from .serializers import BookSerializer

class BookListCreateView(generics.ListCreateAPIView):
    queryset = Book.objects.all()
    serializer_class = BookSerializer

class BookRetrieveUpdateDeleteView(generics.RetrieveUpdateDestroyAPIView):
    queryset = Book.objects.all()
    serializer_class = BookSerializer


update urls.py 

In [ ]:
from django.urls import path
from .views import BookListCreateView, BookRetrieveUpdateDeleteView

urlpatterns = [
    path('books/', BookListCreateView.as_view(), name='book-list-create'),
    path('books/<int:pk>/', BookRetrieveUpdateDeleteView.as_view(), name='book-retrieve-update-delete'),
]


### ViewSets and Routers
ViewSets combine logic for listing, creating, retrieving, updating, and deleting into one class.

In [ ]:
from rest_framework import viewsets
from .models import Book
from .serializers import BookSerializer

class BookViewSet(viewsets.ModelViewSet):
    queryset = Book.objects.all()
    serializer_class = BookSerializer


Define a router in urls.py:

In [ ]:
from rest_framework.routers import DefaultRouter
from .views import BookViewSet

router = DefaultRouter()
router.register(r'books', BookViewSet)

urlpatterns = router.urls


### 5. Adding Pagination
DRF provides built-in support for pagination.

Enable Pagination:

In [ ]:
# settings.py
REST_FRAMEWORK = {
    'DEFAULT_PAGINATION_CLASS': 'rest_framework.pagination.PageNumberPagination',
    'PAGE_SIZE': 10,
}


## 6. Authentication and Permissions
Authentication:

DRF supports several authentication methods:

SessionAuthentication (default)

TokenAuthentication

JWT (JSON Web Token)
Enable Token Authentication:

In [ ]:
pip install djangorestframework-simplejwt


In [ ]:
# Update settings.py:
REST_FRAMEWORK = {
    'DEFAULT_AUTHENTICATION_CLASSES': [
        'rest_framework_simplejwt.authentication.JWTAuthentication',
    ],
}


Update urls.py to include token endpoints:

In [ ]:
from rest_framework_simplejwt.views import TokenObtainPairView, TokenRefreshView

urlpatterns += [
    path('api/token/', TokenObtainPairView.as_view(), name='token_obtain_pair'),
    path('api/token/refresh/', TokenRefreshView.as_view(), name='token_refresh'),
]


## Permissions:
DRF permissions control access to views.

In [ ]:
from rest_framework.permissions import IsAuthenticated

class BookListCreateView(generics.ListCreateAPIView):
    queryset = Book.objects.all()
    serializer_class = BookSerializer
    permission_classes = [IsAuthenticated]


## 7. Testing APIs
Install Postman or Use Curl

Use tools like Postman or curl to test your API endpoints.

Example curl command:

In [ ]:
curl -X GET http://127.0.0.1:8000/books/
